# CS 4650 - Natural Language Understanding - Semester Project
Georgia Tech, Spring 2024 (Instructor: Kartik Goyal)

Group 18

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

c:\Users\pengu\.conda\envs\cs4650_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#parses XML files (4 downloaded files from website: https://zenodo.org/records/1489920)
def parse_xml_articles(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    articles_data = []
    for article in root.findall('article'):
        article_data = {
            'id': article.get('id'),
            'text': ''.join(p.text for p in article.findall('.//p') if p.text)
        }
        articles_data.append(article_data)
    return pd.DataFrame(articles_data)

def parse_xml_labels(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    labels_data = []
    for article in root.findall('article'):
        label_data = {
            'id': article.get('id'),
            'hyperpartisan': article.get('hyperpartisan') == 'true',
            'bias': article.get('bias')
        }
        labels_data.append(label_data)
    return pd.DataFrame(labels_data)

In [3]:
# Replace 'path_to_your_files' with the actual paths to your XML files
# Here is the download link from the website: https://zenodo.org/records/1489920 
# Training dataset and training ground truth is a piar, validation dataset and validation ground truth is a pair (their IDs correspond)
train_articles = parse_xml_articles(r'data\articles-training-bypublisher-20181122.xml')
train_labels = parse_xml_labels(r'data\ground-truth-training-bypublisher-20181122.xml')
validation_articles = parse_xml_articles(r'data\articles-validation-bypublisher-20181122.xml')
validation_labels = parse_xml_labels(r'data\ground-truth-validation-bypublisher-20181122.xml')

In [4]:
# Merge articles and labels on ID
train_data = pd.merge(train_articles, train_labels, on='id')
validation_data = pd.merge(validation_articles, validation_labels, on='id')

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_data)
validation_dataset = Dataset.from_pandas(validation_data)

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("alexgshaw/hyperpartisan-classifier")

#def tokenize_function(examples):
#    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_validation = validation_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 150000/150000 [01:00<00:00, 2459.28 examples/s]


In [7]:
# Tokenization
tokenizer = AutoTokenizer.from_pretrained("alexgshaw/hyperpartisan-classifier")

c:\Users\pengu\.conda\envs\cs4650_project\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pengu\.cache\huggingface\hub\models--alexgshaw--hyperpartisan-classifier. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
# Model
# BERT model specialized for sequence classification tasks (pre-trained)
# The training process below fine-tunes this BERT model (fine tunes its parameteres) to fit "https://huggingface.co/datasets/hyperpartisan_news_detection"
model = AutoModelForSequenceClassification.from_pretrained("alexgshaw/hyperpartisan-classifier")


#for analysis later on
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [6]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy='epoch'
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

  0%|          | 0/112500 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [9]:
# Evaluate the model
evaluation_results = trainer.evaluate()

print(evaluation_results)

predictions_output = trainer.predict(tokenized_validation)
predictions = predictions_output.predictions.argmax(axis=-1)

In [ ]:
# Getting the labels from the dataset
true_labels = np.array(tokenized_validation["labels"])

# Finding where predictions and true labels differ
mislabeled_indices = np.where(predictions != true_labels)[0]

# Analyze mislabeled examples
for index in mislabeled_indices:
    print(f"ID: {validation_data.iloc[index]['id']}")
    print(f"Text: {validation_data.iloc[index]['text']}")
    print(f"True label: {true_labels[index]}, Predicted label: {predictions[index]}")
    print("\n")